# 动物识别

百度AI提供动物识别功能，识别近八千种动物，接口返回动物名称。可以帮助叫出更多动物的名字。

# 开始我们的实验
(1)登录百度AI开发平台，选择“图像识别”，使用“创建应用”建立一个动物识别应用。 获取应用的AppID、API Key、Secret Key，注意:Secret Key要先选择显示，然后复制其现实的字符。

例如我创建的图像识别应用获取的AppID、API Key、Secret Key如下：

    """ 公开课限定量图像识别，需要更改为自己的api接口 """
    pic_APP_ID = '21222675'
    pic_API_KEY = 'kuuuxPDhItuMo1b4eQgipBAV'
    pic_SECRET_KEY = 'ZWzTrLWfQMy6AXT9FU9X5ggHXAfkFU9I'

百度AI开发平台注册链接:https://login.bce.baidu.com/?account=&redirect=http%3A%2F%2Fconsole.bce.baidu.com%2F%3Ffromai%3D1#/aip/overview

(2)熟悉我封装的百度AI图像识别的模块 在代码中输入如下代码，导入我的封装的模块

    import sys
    sys.path.append('../baidu_api_lib')
    from baidu_picture import baidu_picture_2_msg
封装库提供两个方法：初始化方法，图像识别方法。

<1>初始化方法，需要传入三个参数，就是上步我们创建“图像识别”应用获取的三个参数 参数信息如下：

    """ 公开课限定量图像识别，需要更改为自己的api接口 """
    pic_APP_ID = '21222675'
    pic_API_KEY = 'kuuuxPDhItuMo1b4eQgipBAV'
    pic_SECRET_KEY = 'ZWzTrLWfQMy6AXT9FU9X5ggHXAfkFU9I'
    
调用方式：

    # 传入百度AI的参数，进行图像识别
    pic_msg = baidu_picture_2_msg(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)
    初始化完毕后，获取pic_msg对象，操作该对象就能进行动物识别。
    
<2>动物识别方法 这是我封装百度动物识别方法的函数原型：

    #从百度AI获取图片分析结果
    response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg')

baidu_request_url:百度AI的调用接口，

    #百度AI的调用url
    baidu_request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/animal"

    'read_word.jpg':进行动物识别的图片。
    返回值:response为文字识别的信息

    response.json()["words_result"]:识别结果数组
    response.json()["words_result"][i]["name"]:第i个识别结果名称
    response.json()["words_result"][i]["score"]:第i个识别结果概率
    
    备注：
        response.json()["words_result"][0]["name"]为百度AI认为识别最高的动物名称
        response.json()["words_result"][0]["score"为百度AI认为识别最高的动物概率
...还有不少参数，通过print(response.json())可以发现

In [ ]:
######################################################
#
# 先感受下小派认识动物的本领,选择本cell，按shirt+enter键运行本模块
#
######################################################
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright (c) 2020 Taste all Pi.
#
# Licensed under the GNU General Public License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.gnu.org/licenses/gpl-2.0.html
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#导入标准库
import sys
import os
from playsound import playsound
import cv2 as cv
import time
from multiprocessing import Process, Queue
import multiprocessing

#导入自定义库
sys.path.append('../baidu_api_lib')
from baidu_picture import baidu_picture_2_msg
from baidu_sound import baidu_word_2_sound

""" 公开课限定量图像识别，需要更改为自己的api接口 """
pic_APP_ID = '21222675'
pic_API_KEY = 'kuuuxPDhItuMo1b4eQgipBAV'
pic_SECRET_KEY = 'ZWzTrLWfQMy6AXT9FU9X5ggHXAfkFU9I'

#百度AI的调用url
baidu_request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/animal"

#　获取摄像头图像
def camera_frame_func(task_name, mult_queue1, mydict):
    
    # 创建一个VideoCapture对象
    capture = cv.VideoCapture(0) 
    
    #更新人脸识别信息
    mydict["get_pic"] = "false"
    
    #　给出提示信息
    print(task_name + "任务启动")
    
    try:
        while True:
            # 一帧一帧读取视频
            ret, frame = capture.read()
            
            #将拍摄到的图片发送到消息队列中
            if mydict["get_pic"] == "true":
                mydict["get_pic"] = "false"
                mult_queue1.put(frame)

            # 本地显示视频图像
            cv.imshow('real_time picture', frame) 
            cv.waitKey(1)

    except KeyboardInterrupt:
        # 释放cap,销毁窗口
        capture.release()    
        #关闭显示窗口
        cv.destroyAllWindows() 
        print(task_name + "任务被终止")
        
#处理图像
def proc_frame_func(task_name, mult_queue, mydict):
    #　给出提示信息
    print(task_name + "任务启动")
    
    # 传入百度AI的参数，进行图像识别
    pic_msg = baidu_picture_2_msg(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)
    
    #传入百度AI的参数，进行语音合成
    word_2_sound = baidu_word_2_sound(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)
    word_2_sound.trans_word_to_sound("咱们比拼下谁认识的动物多，让我们开始吧",'tst_sound.mp3')
    os.system('mplayer ' + 'tst_sound.mp3')
    time.sleep(2)
    
    try:
       
        while True:
            #给出语音提示
            word_2_sound.trans_word_to_sound("给我3秒时间，让我想想这是什么动物",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(2)
            word_2_sound.trans_word_to_sound("3",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)
            
            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("2",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)

            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("1",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)
            
            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #关闭显示窗口
            cv.destroyAllWindows()            

            # 写入图片
            cv.imwrite('read_word.jpg',frame)
            
            #从百度AI获取图片分析结果
            response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg')
            
            #给出百度AI分析的数据
            print(response.json())
            
            #获取分析结果中的动物名称，并语音播报
            animal = response.json()['result'][0]['name']
            
            if animal == "非动物":
                word_2_sound.trans_word_to_sound("你在跟我开玩笑嘛，这不是动物",'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
            else:
                word_2_sound.trans_word_to_sound("这是"+animal,'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
                
            #给出语音提示
            word_2_sound.trans_word_to_sound("我厉害吧，不服就继续比拼",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')    
            time.sleep(3)
            
    except KeyboardInterrupt:
        os.remove('tst_sound.mp3')
        os.remove('read_word.jpg')
        print(task_name + "任务被终止")
        
if __name__ == "__main__":
    try:
        
        mydict=multiprocessing.Manager().dict()
        
        #　定义传递图像队列和传递图像处理结果队列
        q_frame = Queue()
        
        #　采集摄像头进程、处理图片进程、播报语音信息
        get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))
        proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, mydict))
        
        # 启动任务
        get_camera_frame.start()
        proc_frame.start()

    except KeyboardInterrupt:
        print("任务被终止了")

# 让我们临摹代码，学习“动物识别”功能

In [ ]:
# #导入标准库
# import sys

# import os

# from playsound import playsound

# import cv2 as cv

# import time

# from multiprocessing import Process, Queue

# import multiprocessing


# #导入自定义库
# sys.path.append('../baidu_api_lib')

# from baidu_picture import baidu_picture_2_msg

# from baidu_sound import baidu_word_2_sound


# """ 公开课限定量图像识别，需要更改为自己的api接口 """
# pic_APP_ID = '21222675'

# pic_API_KEY = 'kuuuxPDhItuMo1b4eQgipBAV'

# pic_SECRET_KEY = 'ZWzTrLWfQMy6AXT9FU9X5ggHXAfkFU9I'


# #百度AI的调用url
# baidu_request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/animal"


# #　获取摄像头图像
# def camera_frame_func(task_name, mult_queue1, mydict):

    
#     # 创建一个VideoCapture对象
#     capture = cv.VideoCapture(0) 

    
#     #更新人脸识别信息
#     mydict["get_pic"] = "false"

    
#     #　给出提示信息
#     print(task_name + "任务启动")

    
#     try:

#         while True:

#             # 一帧一帧读取视频
#             ret, frame = capture.read()

            
#             #将拍摄到的图片发送到消息队列中
#             if mydict["get_pic"] == "true":

#                 mydict["get_pic"] = "false"

#                 mult_queue1.put(frame)


#             # 本地显示视频图像
#             cv.imshow('real_time picture', frame) 

#             cv.waitKey(1)


#     except KeyboardInterrupt:

#         # 释放cap,销毁窗口
#         capture.release()    

#         #关闭显示窗口
#         cv.destroyAllWindows() 

#         print(task_name + "任务被终止")

        
# #处理图像
# def proc_frame_func(task_name, mult_queue, mydict):

#     #　给出提示信息
#     print(task_name + "任务启动")

    
#     # 传入百度AI的参数，进行图像识别
#     pic_msg = baidu_picture_2_msg(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)

    
#     #传入百度AI的参数，进行语音合成
#     word_2_sound = baidu_word_2_sound(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)

#     word_2_sound.trans_word_to_sound("咱们比拼下谁认识的动物多，让我们开始吧",'tst_sound.mp3')

#     os.system('mplayer ' + 'tst_sound.mp3')

#     time.sleep(2)

    
#     try:

       
#         while True:

#             #给出语音提示
#             word_2_sound.trans_word_to_sound("给我3秒时间，让我想想这是什么动物",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')

#             time.sleep(2)

#             word_2_sound.trans_word_to_sound("3",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')

#             time.sleep(1)

            
#             #从队列中获取图片，显示图像质量
#             mydict["get_pic"] = "true"

#             frame = mult_queue.get()

#             cv.waitKey(1)

            
#             #给出语音提示
#             word_2_sound.trans_word_to_sound("2",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')

#             time.sleep(1)


#             #从队列中获取图片，显示图像质量
#             mydict["get_pic"] = "true"

#             frame = mult_queue.get()

#             cv.waitKey(1)

            
#             #给出语音提示
#             word_2_sound.trans_word_to_sound("1",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')

#             time.sleep(1)

            
#             #从队列中获取图片，显示图像质量
#             mydict["get_pic"] = "true"

#             frame = mult_queue.get()

#             cv.waitKey(1)

            
#             #关闭显示窗口
#             cv.destroyAllWindows()        


#             # 写入图片
#             cv.imwrite('read_word.jpg',frame)

            
#             #从百度AI获取图片分析结果
#             response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg')

            
#             #给出百度AI分析的数据
#             print(response.json())

            
#             #获取分析结果中的动物名称，并语音播报
#             animal = response.json()['result'][0]['name']

            
#             if animal == "非动物":
#                 word_2_sound.trans_word_to_sound("你在跟我开玩笑嘛，这不是动物",'tst_sound.mp3')

#                 os.system('mplayer ' + 'tst_sound.mp3')

#             else:
#                 word_2_sound.trans_word_to_sound("这是"+animal,'tst_sound.mp3')

#                 os.system('mplayer ' + 'tst_sound.mp3')

                
#             #给出语音提示
#             word_2_sound.trans_word_to_sound("我厉害吧，不服就继续比拼",'tst_sound.mp3')

#             os.system('mplayer ' + 'tst_sound.mp3')   

#             time.sleep(3)

            
#     except KeyboardInterrupt:
#         os.remove('tst_sound.mp3')

#         os.remove('read_word.jpg')

#         print(task_name + "任务被终止")

        
# if __name__ == "__main__":

#     try:
        
#         mydict=multiprocessing.Manager().dict()

        
#         #　定义传递图像队列和传递图像处理结果队列
#         q_frame = Queue()

        
#         #　采集摄像头进程、处理图片进程、播报语音信息
#         get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))

#         proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, mydict))

        
#         # 启动任务
#         get_camera_frame.start()

#         proc_frame.start()


#     except KeyboardInterrupt:

#         print("任务被终止了")


# 课后练习
1、小派不仅认识动物，还认识植物、菜品、还有各国货币。

In [ ]:
######################################################
#
# 先感受下小派认识动物的本领,选择本cell，按shirt+enter键运行本模块
#
######################################################
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright (c) 2020 Taste all Pi.
#
# Licensed under the GNU General Public License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.gnu.org/licenses/gpl-2.0.html
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#导入标准库
import sys
import os
from playsound import playsound
import cv2 as cv
import time
from multiprocessing import Process, Queue
import multiprocessing

#导入自定义库
sys.path.append('../baidu_api_lib')
from baidu_picture import baidu_picture_2_msg
from baidu_sound import baidu_word_2_sound

""" 公开课限定量图像识别，需要更改为自己的api接口 """
pic_APP_ID = '21222675'
pic_API_KEY = 'kuuuxPDhItuMo1b4eQgipBAV'
pic_SECRET_KEY = 'ZWzTrLWfQMy6AXT9FU9X5ggHXAfkFU9I'

#百度AI的调用url，货币识别
baidu_request_url = "https://aip.baidubce.com/rest/2.0/image-classify/v1/currency"

#　获取摄像头图像
def camera_frame_func(task_name, mult_queue1, mydict):
    
    # 创建一个VideoCapture对象
    capture = cv.VideoCapture(0) 
    
    #更新图片识别信息
    mydict["get_pic"] = "false"
    
    #　给出提示信息
    print(task_name + "任务启动")
    
    try:
        while True:
            # 一帧一帧读取视频
            ret, frame = capture.read()
            
            #将拍摄到的图片发送到消息队列中
            if mydict["get_pic"] == "true":
                mydict["get_pic"] = "false"
                mult_queue1.put(frame)

            # 本地显示视频图像
            cv.imshow('real_time picture', frame) 
            cv.waitKey(1)

    except KeyboardInterrupt:
        # 释放cap,销毁窗口
        capture.release()    
        #关闭显示窗口
        cv.destroyAllWindows() 
        print(task_name + "任务被终止")
        
#处理图像
def proc_frame_func(task_name, mult_queue, mydict):
    #　给出提示信息
    print(task_name + "任务启动")
    
    # 传入百度AI的参数，进行图像识别
    pic_msg = baidu_picture_2_msg(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)
    
    #传入百度AI的参数，进行语音合成
    word_2_sound = baidu_word_2_sound(pic_APP_ID, pic_API_KEY, pic_SECRET_KEY)
    word_2_sound.trans_word_to_sound("咱们比拼下谁认识的多，让我们开始吧",'tst_sound.mp3')
    os.system('mplayer ' + 'tst_sound.mp3')
    time.sleep(2)
    
    try:
       
        while True:
            #给出语音提示
            word_2_sound.trans_word_to_sound("给我3秒时间，让我想想这是什么",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(2)
            word_2_sound.trans_word_to_sound("3",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)
            
            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("2",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)

            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #给出语音提示
            word_2_sound.trans_word_to_sound("1",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')
            time.sleep(1)
            
            #从队列中获取图片，显示图像质量
            mydict["get_pic"] = "true"
            frame = mult_queue.get()
            cv.waitKey(1)
            
            #关闭显示窗口
            cv.destroyAllWindows()            

            # 写入图片
            cv.imwrite('read_word.jpg',frame)
            
            #从百度AI获取图片分析结果
            response = pic_msg.pic_2_msg(baidu_request_url, 'read_word.jpg')
            
            #给出百度AI分析的数据
            print(response.json())
            
            #获取分析结果中的图像名称，并语音播报
            thing = response.json()['result']['hasdetail']
            
            if thing:
                #获取货币名称和面值
                currencyName = response.json()['result']['currencyName']
                currencyDenomination = response.json()['result']['currencyDenomination']
                
                #语音播报获取到的信息
                word_2_sound.trans_word_to_sound("这是"+currencyName+'面值'+currencyDenomination,'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
            else:
                word_2_sound.trans_word_to_sound("这不是货币，换个真货币",'tst_sound.mp3')
                os.system('mplayer ' + 'tst_sound.mp3')
                
            #给出语音提示
            word_2_sound.trans_word_to_sound("我厉害吧，不服就继续比拼",'tst_sound.mp3')
            os.system('mplayer ' + 'tst_sound.mp3')    
            time.sleep(3)
            
    except KeyboardInterrupt:
        os.remove('tst_sound.mp3')
        os.remove('read_word.jpg')
        print(task_name + "任务被终止")
        
if __name__ == "__main__":
    try:
        
        mydict=multiprocessing.Manager().dict()
        
        #　定义传递图像队列和传递图像处理结果队列
        q_frame = Queue()
        
        #　采集摄像头进程、处理图片进程、播报语音信息
        get_camera_frame = Process(target=camera_frame_func, args=("获取摄像头图像", q_frame, mydict))
        proc_frame       = Process(target=proc_frame_func, args=("处理图像", q_frame, mydict))
        
        # 启动任务
        get_camera_frame.start()
        proc_frame.start()

    except KeyboardInterrupt:
        print("任务被终止了")